# Neural network

Modelo de red neuronal que sirve para realizar la prediccion y analizar resultados.

- Lee los datos del df conjunto.
- Lee los datos que se usarán de test.
- Crea el modelo.
- Realiza la predicción.
- Analiza resultados.

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

df = pd.read_parquet('data/data.parquet')
test_data = pd.read_parquet("test/submission.parquet")

In [ ]:
df.head(5)

### Selección de características

In [ ]:
#Variables globales
seed = 42
batch_size = 32
epochs = 10

features = ['station_id','hour','dayofweek','festa','month','Rain','Wind','ctx-1','ctx-2','ctx-3','ctx-4']
target = ['porcio']

### Dividir en train y test, normalizar datos y PCA

In [ ]:
# train_test_split
X = df[features]
y = df[target]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=seed)

X_test = test_data[features]
y_test = test_data[target]

# Normalizar los datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Aplicar Análisis de Componentes Principales, manteniendo explicada el 95% de la varianza
pca = PCA(n_components=0.95)  
X_train_pca = pca.fit_transform(X_train_scaled)
X_val_pca = pca.transform(X_val_scaled)
X_test_pca = pca.transform(X_test_scaled)

### Definicion de la red neuronal, compilacion y entrenamiento del modelo

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1)
])

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=1000,
    decay_rate=0.9
)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

model.compile(optimizer=optimizer, loss='mean_squared_error')

model.fit(X_train_pca, y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(X_val_pca, y_val))

### Predicción

In [ ]:
# Prediccion
predictions = model.predict(X_test_pca)

### Evaluación

In [ ]:
# Evaluación
score = model.evaluate(X_test_pca, y_test, verbose=0)
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)
print("Pérdida en el conjunto de prueba:", score)
print('MSE:', mse)
print('R^2:', r2)

### Comparar las predicciones con los valores reales

In [ ]:
df_predictions = pd.DataFrame({'Predicciones': predictions.flatten(), 'Valores Reales': y_test.values.flatten()})
df_predictions

### Análisis descriptivo

In [ ]:
describe_results = df_predictions.describe()
describe_results

### Visualización de resultados

In [ ]:
import matplotlib.pyplot as plt

# Graficar las predicciones frente a las etiquetas reales
plt.scatter(y_test, predictions)
plt.xlabel('Etiquetas reales')
plt.ylabel('Predicciones')
plt.title('Comparación entre las etiquetas reales y las predicciones')
plt.show()